### Universidad de Burgos - Algoritmia - Curso 2022/2023
---
# Práctica Obligatoria 4 - Métodos voraces
*6 Créditos (15 horas). Peso en la calificación final: 10%*

---

#### Autor: 
- Álvaro Manjón Vara



Eres un programador freelance que ofrece servicios de optimización de procesos para empresas. Te han llegado dos ofertas de trabajo que no puedes rechazar. Uno es de la red de restaurantes **Sopas y Caldos S.L.**, especializada en sopas, caldos y estofados; y el otro es del **Ministerio de Transporte, Movilidad y Agenda Urbana** para desarrollar un plan de carreteras para conectar varias comarcas.
# 1. Problemas a resolver
Resuelve los siguientes problemas sobre este mismo notebook. Implementa las funciones indicadas con _pass_.

## 1.1 Optimizar producción de sopas

Sopas y Caldos S.L. se plantea optimizar la producción de ollas de sopas, caldos y estofados para cada uno de sus restaurantes. Se pide:

1. Dado el presupuesto total diario del restaurante, el índice de popularidad de cada plato (especificada en el registro total de comandas recibidas en la historia del restaurante) y el precio de producción de cada olla, se pide calcular que ollas de sopa (o partes de ella) se han de producir para optimizar la popularidad total de las sopas.

Cada olla de sopa se hace una sola vez y se puede hacer una parte de olla (entre 0 y 1) siendo 0 que no se hace esa sopa y 1 que la olla se hace entera. El coste de cada sopa es proporcional a la cantidad que se hace.

## 1.2 Plan de carreteras

A partir de una red de localidades se te da el coste de hacer una carretera entre cada par. Crear una red que minimice el coste de construcción con dos condiciones:
1. No puede haber zonas aisladas, desde cada localidad se debe poder llegar a cualquier otra por toda la red de carreteras.
2. Hay que minimizar costes, por lo que no puede haber ciclos cerrados en la red. E.g si la ciudad 'A' está conectada a la ciudad 'B' y la ciudad 'B' a la ciudad 'C', no se ha de construir una carretera que conecte de la ciudad 'A' a la 'C'.

---
**Recuerda**
* Solamente puedes utilizar librerías nativas (https://docs.python.org/es/3.7/library/index.html).
* Las funciones que importes no son "gratis", cada una tendrá una complejidad temporal y espacial que se tendrá que tener en cuenta.

**Entrega**
* Nombra el fichero a entregar como: `<apellidos>_voraz.ipynb`.
* Verifica que la entrega no está corrupta. En caso de que el archivo entregado no pueda abrirse, se evaluará con **0 puntos**.

# 2. Preguntas sobre la actividad
Tras resolver los problemas planteados, responde a las preguntas que se plantean al final de este notebook.

---

# Rúbrica

### Código (6/10)

**Sopas y Caldos S.L. (50%)**
* Funcionalidad correcta: 7/10 puntos
* Complejidad temporal de la solución: 2/10 puntos
* Complejidad espacial de la solución: 1/10 puntos

**Ministerio de Transporte, Movilidad y Agencia Urbana (50%)**
* Funcionalidad correcta: 6/10 puntos
* Complejidad temporal de la solución: 1.5/10 puntos
* Complejidad espacial de la solución: 2.5/10 puntos

### Respuesta a las preguntas (4/10)
* Análisis temporal y espacial: 4/10 puntos
* Respuesta al resto de cuestiones planteadas: 6/10 puntos


In [39]:
# Imports
import heapq

### **Sopas y Caldos S.L.**

In [40]:
class Sopa:
    """
    Clase Sopa. 
    Representa una sopa o caldo.
    """    
    
    def __init__(self, nombre, coste):
        """Crea un objeto de clase Sopa

        Parameters
        ----------
        nombre : str
            Nombre de la sopa
        coste : number
            Coste de hacer una olla de sopa
        """
        self.nombre = nombre
        self.coste = coste
        self.popularidad = {}

    
    def __hash__(self):
        """Genera el valor hash identificativo de la sopa

        Returns
        -------
        int
            Valor hash
        """        
        return hash((self.nombre, self.coste))
        
    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """        
        return f"La sopa {self.nombre} tiene coste {self.coste}"
        
    def __repr__(self):
        """Genera una cadena descriptiva del objeto dentro de colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """  
        return f"Sopa({self.nombre}, {self.coste})"
               
    def set_popularidad(self, restaurante, pedidos):
        """Dado un restuarante y un número de pedidos anteriores,
           almacena para esta sopa la popularidad que tiene.

        Parameters
        ----------
        restaurante : str
            Restaurante donde se ha pedido esta sopa o caldo
        pedidos : int
            Número de pedidos anteriores
        """        
        self.popularidad[restaurante] = pedidos
            
    def get_popularidad(self, restaurante):
        """Dado un restaurante, obtiene la popularidad de la sopa

        Parameters
        ----------
        restaurante : str
            Restaurante donde se vende esta sopa o caldo

        Returns
        -------
        int
            Popularidad de la sopa en el restaurante
        """        
        return self.popularidad[restaurante]
        
    """Funciones del profesor"""
    def __eq__(self, obj):
        return self.nombre == obj.nombre and self.coste == obj.coste
    
    def __lt__(self, obj):
        return self.coste < obj.coste

In [41]:
class Restaurante:
    """
    Clase del restuarante de Sopas y Caldos SL.
    """
    
    def __init__(self, identificador, nombre, presupuesto):
        """Instancia un Restaurante

        Parameters
        ----------
        identificador : int
            Valor que identifica el restaurante.
        nombre : str
            Nombre del restaurante.
        presupuesto : int
            Cantidad de dinero para hacer las ollas de sopa en un día.
        """
        self.identificador = identificador
        self.nombre = nombre
        self.presupuesto = presupuesto
        self.preparacion_sopas = {}
        
    def __hash__(self):
        """Genera el valor hash identificativo del restaurante

        Returns
        -------
        int
            Valor hash
        """     
        return ((self.identificador, self.nombre, self.presupuesto))
        
    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """      
        return f"El restaurante {self.nombre} tiene presupuesto {self.presupuesto}"
        
    def __repr__(self):
        """Genera una cadena descriptiva del objeto en colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """      
        return f"Restaurante({self.identificador}, {self.nombre}, {self.presupuesto})"
            
    def prepara(self, sopas):
        """Dada una colección de sopas o caldos,
           seleccionar de cada uno cuanta cantidad (entre 0 y 1)
           se van a preparar.
           Se ha de optimizar la popularidad para que sea la máxima posible.

        Parameters
        ----------
        sopas : collection
            Colección de sopas que se pueden preparar.
        """
        self.preparacion_sopas = {i:0 for i in sopas}
        coste_total = 0
        seleccion_sopas = {}

        for sopa in sopas:
            seleccion_sopas[sopa] = sopa.get_popularidad(self.nombre) / sopa.coste

        while coste_total < self.presupuesto and len(seleccion_sopas) > 0:
            sopa_optima = max(seleccion_sopas, key=seleccion_sopas.get)

            if coste_total + sopa_optima.coste <= self.presupuesto:
                self.preparacion_sopas[sopa_optima] = 1
                coste_total += sopa_optima.coste
            else:
                self.preparacion_sopas[sopa_optima] = (self.presupuesto - coste_total) / sopa_optima.coste
                coste_total = self.presupuesto
                
            seleccion_sopas.pop(sopa_optima)

        return coste_total
            
    def cantidad_preparada(self, sopa):
        """Obtiene la cantidad de sopa preparada en el restaurante.

        Parameters
        ----------
        sopa : Sopa object
            Sopa del cual se quiere saber la cantidad preparada

        Returns
        -------
        float
            Cantidad de sopa preparada en el restaurante.          
        """ 
        return self.preparacion_sopas[sopa]
    
    def sopas_preparadas(self):
        """Sopas preparadas en el restaurante

        Returns
        -------
        set
            Conjunto de tuplas (sopa, cantidad) de las sopas PREPARADAS en el restaurante.          
        """ 
        sopas_preparadas_set = set()

        for sopa in self.preparacion_sopas:
            if self.preparacion_sopas[sopa] != 0:
                sopas_preparadas_set.add((sopa, self.preparacion_sopas[sopa]))

        return sopas_preparadas_set
                
    
    def popularidad_total(self):
        """A partir de las sopas preparadas
           devolver la popularidad proporcional a la cantidad de sopa preparada.

        Returns
        -------
        number
            Popularidad proporcional a la cantidad de sopa preparada.            
        """        
        popularidad = 0

        for sopa in self.preparacion_sopas:
            popularidad += sopa.get_popularidad(self.nombre) * self.preparacion_sopas[sopa]

        return popularidad
    
    

### **Ministerio de Transporte, Movilidad y Agenda Urbana**

In [42]:
class RedComarcas:
    """
    Red de comarcas para conectar con carreteras.
    """
    
    def __init__(self, comarcas, costes):
        """Instancia de la red de comarcas

        Parameters
        ----------
        comarcas : Iterable
            Conjunto de Comarcas disponibles
        distancias : dict{str (Nombre comarca): dict{str (Nombre comarca): int}}
            Grafo de costes de una carretera entre dos comarcas.
        """        
        self.comarcas = set(comarcas)
        self.costes = costes
        self.carreteras = {}        
          
    def construir_carreteras(self):
        """A partir del grafo de costes
           hacer una selección de carreteras que conecten todas las comarcas
           con el menor coste posible. Todas las comarcas deben estar conectadas a la red.
        """
        monticulo = []
        nodos_visitados = set()
        carreteras_obtenidas = {}

        nodo_inicial = list(self.comarcas)[0]
        nodos_visitados.add(nodo_inicial)

        for nodo_destino, coste in self.costes[nodo_inicial].items():
            if nodo_destino != nodo_inicial:
                heapq.heappush(monticulo, (coste, nodo_inicial, nodo_destino))

        while monticulo:
            coste, nodo_origen, nodo_destino = heapq.heappop(monticulo)

            if nodo_destino not in nodos_visitados:
                nodos_visitados.add(nodo_destino)

                if nodo_origen not in carreteras_obtenidas:
                    carreteras_obtenidas[nodo_origen] = {}
                carreteras_obtenidas[nodo_origen][nodo_destino] = coste

                for nodo_siguiente, coste_siguiente in self.costes[nodo_destino].items():
                    if nodo_siguiente not in nodos_visitados:
                        heapq.heappush(monticulo, (coste_siguiente, nodo_destino, nodo_siguiente))


        self.carreteras = carreteras_obtenidas

    def get_grafo(self):
        """Devuelve el grafo de costes recibido

        Returns
        -------
        dict{str (Nombre comarca): dict{str (Nombre comarca): int}}
            Grafo de costes de una carretera entre dos comarcas.
        """        
        return self.costes

    def get_carreteras(self):
        """Devuelve el grafo de carreteras a construir

        Returns
        -------
        Iterable
            Grafo de carreteras a construir
        """        
        return self.carreteras

    def coste_total(self):
        """Devuelve el coste total de las carreteras a construir

        Returns
        -------
        int
            Coste total de las carreteras a construir
        """        
        coste = 0

        for i in self.carreteras.values():
            for j in i.values():
                coste += j

        return coste

#### Caso de ejemplo

In [43]:
import json
import unittest

def carga_dataset(data):
    with open(data) as f:
        test_datasets = json.load(f)

    sopas = list()
    for comarca in test_datasets["sopas"]:
        s_obj = Sopa(comarca["nombre"], comarca["coste"])
        for c, u in comarca["popularidad"].items():
            s_obj.set_popularidad(c, u)
        sopas.append(s_obj)

    restaurantes = dict()
    for comarca in test_datasets["restaurantes"]:
        restaurantes[comarca["nombre"]] = Restaurante(comarca["identificador"], comarca["nombre"], comarca["presupuesto"])


    carreteras = test_datasets["carreteras"]
    c_ = dict()
    for comarca in carreteras:
        c_[comarca] = dict()
        for c in carreteras[comarca]:
            c_[comarca][c] = carreteras[comarca][c]

    red = RedComarcas(carreteras.keys(), c_)

    return sopas, restaurantes, red


**Ejemplo de test básico**

---
**CUIDADO**: Este test trivial se proporciona a modo de ejemplo. **Superar este test no implica, ni mucho menos, que la solución a los ejercicios esté correctamente diseñada y/o implementada.**
El alumno puede incluir sus propios test y conjuntos de datos, derivados de los enunciados, para comprobar la corrección de la solución.



In [44]:
class TestBasico(unittest.TestCase):
    
    def test_ejemplo(self):
        
        s, r, c = carga_dataset("toy.json")

        restaurante_1 = r["El pintor"]
        restaurante_1.prepara(s)
        self.assertEqual(restaurante_1.popularidad_total(), 15.8)
        preparadas = restaurante_1.sopas_preparadas()
        self.assertIn((Sopa("Sopa de castañas", 5), 0.8), preparadas)
        
        c.construir_carreteras()
        self.assertEqual(c.coste_total(), 30)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### **Preguntas sobre la actividad**
Contesta a las siguientes preguntas.

#### **Análisis de la complejidad**

1. Método `Restaurante.prepara`
    * **Complejidad temporal**: _contesta aquí justificando la respuesta_
    * **Complejidad espacial**: _contesta aquí justificando la respuesta_
    
2. Método `ResComarcas.construir_carreteras`
    * **Complejidad temporal**: _contesta aquí justificando la respuesta_
    * **Complejidad espacial**: _contesta aquí justificando la respuesta_

#### **Sopas y Caldos S.L.**

* ¿La solución es óptima (maximiza siempre el ratio de popularidad) o es aproximada?

_Contesta aquí justificando la respuesta_

* ¿Qué efectos a largo plazo tendría utilizar el criterio de popularidad para la planificación? Ten en cuenta que la popularidad se calcula como todas las veces que esa tortilla se ha pedido en el pasado.

_Contesta aquí justificando la respuesta_

#### **Ministerio de Transporte, Movilidad y Agenda Urbana**

* ¿La solución es óptima (minimiza siempre el valor pedido) o es aproximada?

_Contesta aquí justificando la respuesta_

* Se te da un coste por cada par de localidades, pero, si existiesen carreteras que no se puede hacer, ¿qué efectos tendría en tu solución? 

_Contesta aquí justificando la respuesta_